# DataCenter Storage Fail Prediction using RNN(LSTM) model 

### Import all packages used at below codes 

In [68]:
from datetime import datetime

import numpy as np 
import pandas as pd 
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler 

from numpy import array 
from keras.models import Sequential 
from keras.layers import Dense, LSTM 
from keras.layers import Dropout, BatchNormalization, Activation 

import tensorflow as tf 
from keras.models import load_model 


from csv import writer 
from csv import reader 
 

### Input & output file name 

In [121]:
input_file_train = 'train_sample.csv' 
input_file_testA = 'testA_sample.csv' 
input_file_testB = 'testB_sample.csv' 
input_file_problem_A = 'testA_problem.csv' 
input_file_problem_B = 'testB_problem.csv' 

from datetime import datetime
postfix = datetime.now().strftime("%Y%m%d_%H%M%S")

output_file_problem_A = 'output_A_' + postfix + '.csv' 
output_file_problem_B = 'output_B_' + postfix + '.csv' 

checkpoint_filename = 'my_model_checkpoint_' + postfix + '.h5' 
my_model_filename = 'my_model_' + postfix +'.h5' 

### Read input csv files 

In [123]:
import numpy as np 
import pandas as pd 
 

use_feature_Cols = [0,1,2,3,4,5,6,7,8,10,11,12,13,14,19,20] #choose use columns except 0, 1, 2, 20 
#exclude columns: 9, 15, 16, 17, 18 

df_train = pd.read_csv(input_file_train, usecols=use_feature_Cols) 
df_testA = pd.read_csv(input_file_testA, usecols=use_feature_Cols[0:-1]) 
df_testB = pd.read_csv(input_file_testB, usecols=use_feature_Cols[0:-1]) 

### Transform df to array 

In [124]:
X_train_all = np.array(df_train.iloc[:,0:-1]) 
X_testA_all = np.array(df_testA.iloc[:,0:]) 
X_testB_all = np.array(df_testB.iloc[:,0:]) 

### Aggregation all data for scaling(normalizing) 

In [125]:
num_of_data_train = X_train_all.shape[0] 
num_of_data_testA = X_testA_all.shape[0] 
num_of_data_testB = X_testB_all.shape[0] 
num_of_data_all = num_of_data_train + num_of_data_testA + num_of_data_testB 

exclude = [1, 2] 
num_of_features = X_train_all.shape[1] - len(exclude) 

X_train_exclude = np.zeros((num_of_data_train, num_of_features)) 
X_testA_exclude = np.zeros((num_of_data_testA, num_of_features)) 
X_testB_exclude = np.zeros((num_of_data_testB, num_of_features)) 
X_all_exclude = np.zeros((num_of_data_all,num_of_features)) 

i = 0 
for j in range(len(X_train_all)): 
    X_train_exclude[j] = np.delete(X_train_all[j], exclude) 
    X_all_exclude[i] = np.delete(X_train_all[j], exclude) 
    i += 1 

for j in range(len(X_testA_all)): 
    X_testA_exclude[j] = np.delete(X_testA_all[j], exclude) 
    X_all_exclude[i] = np.delete(X_testA_all[j], exclude) 
    i += 1 

for j in range(len(X_testB_all)): 
    X_testB_exclude[j] = np.delete(X_testB_all[j], exclude) 
    X_all_exclude[i] = np.delete(X_testA_all[j], exclude) 
    i += 1 


### Scaling library 

In [126]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler 

mm = MinMaxScaler() 
ss = StandardScaler() 
rs = RobustScaler(quantile_range=(10.0,90.0)) 

#choose standard scaler: transform data variation to -1 < data < 1 
#scaler = mm 
scaler = ss 
#scaler = rs 

X_all_exclude_scaled = scaler.fit_transform(X_all_exclude) 

### Scaling all data 

In [127]:
X_train_exclude_scaled = scaler.transform(X_train_exclude) 
X_testA_exclude_scaled = scaler.transform(X_testA_exclude) 
X_testB_exclude_scaled = scaler.transform(X_testB_exclude) 

### Preprocess data transform 2d array to 3d array (for input of LSTM model) – 1/2 

In [76]:
num_of_serial_number = 0 
max_num_of_data_per_samples = 0 

def CountNumDataAndSamples(X_original): 
    global num_of_serial_number 
    global max_num_of_data_per_samples 
    num_of_data_per_samples = 0 
    for i in range(len(X_original)): 
        num_of_data_per_samples += 1 
        if i == len(X_original) - 1: 
            num_of_serial_number += 1 
        else: 
            if not X_original[i][0] == X_original[i+1][0]: 
                num_of_serial_number += 1 
                if max_num_of_data_per_samples < num_of_data_per_samples: 
                    max_num_of_data_per_samples = num_of_data_per_samples 
                num_of_data_samples = 0 

CountNumDataAndSamples(X_train_exclude_scaled) 
print("num_of_serial_number:", num_of_serial_number)
print("max_num_of_data_per_samples:", max_num_of_data_per_samples) 


num_of_serial_number: 2
max_num_of_data_per_samples: 5


### Preprocess data transform 2d array to 3d array – 2/2 

In [77]:
X_original = X_train_exclude_scaled 

df_serial_number_only = df_train.iloc[:,0] 
df_label = df_train.iloc[:,-1] 
y_original = np.array(df_label) 

i = num_of_serial_number 
j = max_num_of_data_per_samples 
num_use_features = X_original.shape[1] 
k = num_use_features 

num_total_data = X_original.shape[0] 
print("num_of_serial_number:", num_of_serial_number) 
print("max_num_of_data_per_samples:",max_num_of_data_per_samples) 
print("num_use_features:",num_use_features) 
print("num_total_data:",num_total_data) 

X = np.zeros((i, j, k)) 
y = np.zeros((i,1)) 

serial_number = df_serial_number_only[0] 
serial_number_idx = 0 
per_data_idx = 0 
num_class_zero = 0 
num_class_one = 0 

for idx in range(num_total_data): 
    if not df_serial_number_only[idx] == serial_number: 
        serial_number = df_serial_number_only[idx] 
        y[serial_number_idx][0] = y_original[idx-1] 
        if y_original[idx-1] == 0: 
            num_class_zero += 1 
        else: 
            num_class_one += 1 
        serial_number_idx += 1 
        per_data_idx = 0 
    X[serial_number_idx][per_data_idx] = X_original[idx] 
    per_data_idx += 1 

#last Y 
idx += 1 
y[serial_number_idx][0] = y_original[idx-1] 
if y_original[idx-1] == 0: 
    num_class_zero += 1 
else: 
    num_class_one += 1 

print("num_class_zero:", num_class_zero) 
print("num_class_one:", num_class_one) 
 

num_of_serial_number: 2
max_num_of_data_per_samples: 5
num_use_features: 13
num_total_data: 10
num_class_zero: 1
num_class_one: 1


### Oversampling for label 1 

In [83]:
i = num_class_zero * 2 

X_resample = np.zeros((i, j, k)) 
y_resample = np.zeros((i,1)) 

new_idx = 0 
for idx in range(num_of_serial_number): 
    if y[idx] == 0: 
        X_resample[new_idx] = X[idx] 
        y_resample[new_idx] = y[idx] 
        new_idx += 1 

new_idx = num_class_zero 

end = 1 
while end: 
    for idx in range(num_of_serial_number): 
        if y[idx] == 1: 
            X_resample[new_idx] = X[idx] 
            y_resample[new_idx] = y[idx] 
            new_idx += 1 

        if new_idx >= (num_class_zero * 2 - 1): 
            end = 0 
            break 

print("oversampling done") 

oversampling done


### Make LSTM model

In [89]:
from numpy import array 
from keras.models import Sequential 
from keras.layers import Dense, LSTM 
from keras.layers import Dropout, BatchNormalization, Activation 

model = Sequential() 
num_of_output_LSTM = 16 

model.add(LSTM(num_of_output_LSTM, return_sequences=True, input_shape=(max_num_of_data_per_samples, num_use_features))) 
model.add(LSTM(num_of_output_LSTM, return_sequences=True)) 
model.add(LSTM(num_of_output_LSTM)) 
model.add(Dropout(0.1)) 

model.add(Dense(64)) 
model.add(BatchNormalization()) 
model.add(Activation('relu')) 

model.add(Dense(32)) 
model.add(BatchNormalization()) 
model.add(Activation('relu')) 

model.add(Dense(2)) 
model.add(Activation('softmax')) 

model.summary() 

model.compile(loss='SparseCategoricalCrossentropy', optimizer='adam', metrics=['accuracy']) 


Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 5, 16)             1920      
_________________________________________________________________
lstm_5 (LSTM)                (None, 5, 16)             2112      
_________________________________________________________________
lstm_6 (LSTM)                (None, 16)                2112      
_________________________________________________________________
dropout_1 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 64)                1088      
_________________________________________________________________
batch_normalization_2 (Batch (None, 64)                256       
_________________________________________________________________
activation_3 (Activation)    (None, 64)               

### Define checkpoint callback function to save temperal result 

In [91]:
import tensorflow as tf 

callback_checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_filename, monitor='val_loss', verbose=1, save_best_only=True, mode='auto') 


### Define decaying learning rate callback function

In [93]:
import tensorflow as tf 

def scheduler(epoch, lr): 
    print("learning rate:", lr) 
    return (0.01) * ((1/2) ** (epoch / 10)) 

callback_decay_lr = tf.keras.callbacks.LearningRateScheduler(scheduler) 

### define early stopping callback function 

In [94]:
import tensorflow as tf 

epoch = 40 

callback_early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=epoch/10)  

### model training 

In [95]:

model.fit(X_resample, y_resample, epochs=epoch, batch_size=128, shuffle=True, validation_split=0.1, callbacks=[callback_decay_lr, callback_early_stop, callback_checkpoint]) 
 

Epoch 1/40
learning rate: 0.0010000000474974513
1/1 [==============================] - 5s 5s/step - loss: 0.6931 - accuracy: 1.0000 - val_loss: 0.6833 - val_accuracy: 1.0000

Epoch 00001: val_loss improved from inf to 0.68326, saving model to my_model_checkpoint_20210522_134615.h5
Epoch 2/40
learning rate: 0.009999999776482582
1/1 [==============================] - 0s 34ms/step - loss: 0.6832 - accuracy: 1.0000 - val_loss: 0.6741 - val_accuracy: 1.0000

Epoch 00002: val_loss improved from 0.68326 to 0.67414, saving model to my_model_checkpoint_20210522_134615.h5
Epoch 3/40
learning rate: 0.009330329485237598
1/1 [==============================] - 0s 34ms/step - loss: 0.6740 - accuracy: 1.0000 - val_loss: 0.6657 - val_accuracy: 1.0000

Epoch 00003: val_loss improved from 0.67414 to 0.66569, saving model to my_model_checkpoint_20210522_134615.h5
Epoch 4/40
learning rate: 0.008705505169928074
1/1 [==============================] - 0s 33ms/step - loss: 0.6655 - accuracy: 1.0000 - val_loss:


Epoch 00029: val_loss improved from 0.57663 to 0.57554, saving model to my_model_checkpoint_20210522_134615.h5
Epoch 30/40
learning rate: 0.0014358729822561145
1/1 [==============================] - 0s 36ms/step - loss: 0.5731 - accuracy: 1.0000 - val_loss: 0.5745 - val_accuracy: 1.0000

Epoch 00030: val_loss improved from 0.57554 to 0.57453, saving model to my_model_checkpoint_20210522_134615.h5
Epoch 31/40
learning rate: 0.001339716836810112
1/1 [==============================] - 0s 34ms/step - loss: 0.5720 - accuracy: 1.0000 - val_loss: 0.5736 - val_accuracy: 1.0000

Epoch 00031: val_loss improved from 0.57453 to 0.57355, saving model to my_model_checkpoint_20210522_134615.h5
Epoch 32/40
learning rate: 0.0012499999720603228
1/1 [==============================] - 0s 32ms/step - loss: 0.5709 - accuracy: 1.0000 - val_loss: 0.5727 - val_accuracy: 1.0000

Epoch 00032: val_loss improved from 0.57355 to 0.57270, saving model to my_model_checkpoint_20210522_134615.h5
Epoch 33/40
learning r

### load model from checkpoint file 

In [96]:
from keras.models import load_model 

model_last = model 

model = load_model(checkpoint_filename) 


### save model for next time 

In [97]:
from keras.models import load_model 

model.save(my_model_filename) 

reconstructed_model = load_model(my_model_filename) 


### prediction for testA 

### data preprocessing for prediction testA 1/2 

In [128]:
num_of_serial_number = 0 

CountNumDataAndSamples(X_testA_exclude_scaled) 
print("num_of_serial_number:", num_of_serial_number) 
print("max_num_of_data_per_samples:", max_num_of_data_per_samples) 

num_of_serial_number: 2
max_num_of_data_per_samples: 5


### Data preprocessing for prediction testA – 2/2 

In [129]:
X_original = X_testA_exclude_scaled 

df = df_testA 
df_serial_number_only = df.iloc[:, 0] 

df_label = df.iloc[:, -1] 
y_original = np.array(df_label) 

i = num_of_serial_number 
j = max_num_of_data_per_samples 
num_use_features = X_original.shape[1] 
k = num_use_features 

num_total_data = X_original.shape[0] 

print("num_of_serial_number:", num_of_serial_number)
print("max_num_of_data_per_samples:", max_num_of_data_per_samples)
print("num_use_features:", num_use_features)
print("num_total_data:", num_total_data)
 
X = np.zeros((i, j, k)) 
y = np.zeros((i, 1)) 

serial_number = df_serial_number_only[0] 
serial_number_idx = 0 
per_data_idx = 0 

for idx in range(num_total_data): 
    if not df_serial_number_only[idx] == serial_number: 
        serial_number = df_serial_number_only[idx] 
        y[serial_number_idx][0] = y_original[idx-1]
        serial_number_idx += 1 
        per_data_idx = 0 
    X[serial_number_idx][per_data_idx] = X_original[idx] 
    per_data_idx += 1 

#last Y 
idx += 1 
y[serial_number_idx][0] = y_original[idx-1] 


num_of_serial_number: 2
max_num_of_data_per_samples: 5
num_use_features: 13
num_total_data: 10


### Prediction for testA 

In [130]:
X_testA_for_validation = X 

yhat = model.predict(X_testA_for_validation) 
print(yhat.shape) 

num_zero = 0 
num_one = 0 
y_result = np.zeros((num_of_serial_number, 1), dtype=int) 
for i in range(len(yhat)): 
    if yhat[i][0] >= yhat[i][1]: 
        y_result[i][0] = 0
        num_zero += 1 
    else: 
        y_result[i][0] = 1 
        num_one += 1 

y_result_variation = np.zeros((10,2),dtype=int) 

for j in range(len(y_result_variation)): 
    y_result_variation[j][0] = j 

for i in range(len(yhat)): 
    for j in range(len(y_result_variation)): 
        if yhat[i][0] >= 0.1 * y_result_variation[j][0]: 
            if j == 9: 
                y_result_variation[j][1] += 1 
                continue 
            if yhat[i][0] < 0.1 * y_result_variation[j+1][0]: 
                y_result_variation[j][1] += 1 

print(y_result_variation) 

print(y_result.shape) 
print("my_result:", num_zero, "/", num_one) 
print("\n0.9792: 2494 / 461") 
print("0.9857: 2490 / 465") 

y_result_A = y_result 

(2, 2)
[[0 0]
 [1 0]
 [2 0]
 [3 0]
 [4 0]
 [5 2]
 [6 0]
 [7 0]
 [8 0]
 [9 0]]
(2, 1)
my_result: 2 / 0

0.9792: 2494 / 461
0.9857: 2490 / 465


### Define output file save function 

In [134]:
from csv import writer 
from csv import reader 

def add_column_in_csv(input_file, output_file, y_result): 
    with open(input_file, 'r')  as read_obj, \
        open(output_file, 'w', newline='') as write_obj: 
            csv_reader = reader(read_obj) 
            csv_writer = writer(write_obj) 
            init_line = 0 
            count = 0 
            for row in csv_reader: 
                if init_line == 0: 
                    csv_writer.writerow(row) 
                    init_line = 1 
                    continue 
                else:
                    del row[-1]
                    row.append(y_result[count][0]) 
                    csv_writer.writerow(row) 
                    count += 1 
  

### Save output file include test A prediction result 

In [138]:

add_column_in_csv(input_file_problem_A, output_file_problem_A, y_result_A) 


In [ ]:
 

Same for testB 
